<a href="https://colab.research.google.com/github/JuWanMaeng/papaer-review/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [104]:
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,Flatten
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [105]:
(x_train,y_train),(x_test,y_test)=keras.datasets.cifar10.load_data()

In [106]:
x_train=x_train
x_test=x_test

In [107]:
CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [108]:
print(x_train.shape,x_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3)


In [109]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.1)

In [110]:
train_ds=tf.data.Dataset.from_tensor_slices((x_train,y_train))
test_ds=tf.data.Dataset.from_tensor_slices((x_test,y_test))
val_ds=tf.data.Dataset.from_tensor_slices((x_val,y_val))

In [111]:
print(x_train[0].shape)

(32, 32, 3)


In [112]:
def process_images(image,label):
  image=tf.image.resize(image,(224,224))
  return image,label

In [113]:
train_ds=train_ds.batch(batch_size=64)
test_ds=test_ds.batch(batch_size=64)
val_ds=val_ds.batch(batch_size=64)

In [114]:
train_ds=train_ds.map(process_images)
test_ds=test_ds.map(process_images)
val_ds=val_ds.map(process_images)

In [115]:
class VGG16(Sequential):
  def __init__(self):
    super().__init__()

    self.add(Conv2D(64,3,1,padding='same',activation='relu',input_shape=(224,224,3)))
    self.add(Conv2D(64,3,1,padding='same',activation='relu'))

    self.add(MaxPool2D(2,2))
    self.add(Conv2D(128,3,1,padding='same',activation='relu'))
    self.add(Conv2D(128,3,1,padding='same',activation='relu'))

    self.add(MaxPool2D(2,2))
    self.add(Conv2D(256,3,1,padding='same',activation='relu'))
    self.add(Conv2D(256,3,1,padding='same',activation='relu'))
    self.add(Conv2D(256,3,1,padding='same',activation='relu'))

    self.add(MaxPool2D(2,2))
    self.add(Conv2D(512,3,1,padding='same',activation='relu'))
    self.add(Conv2D(512,3,1,padding='same',activation='relu'))
    self.add(Conv2D(512,3,1,padding='same',activation='relu'))

    self.add(MaxPool2D(2,2))
    self.add(Conv2D(512,3,1,padding='same',activation='relu'))
    self.add(Conv2D(512,3,1,padding='same',activation='relu'))
    self.add(Conv2D(512,3,1,padding='same',activation='relu'))

    self.add(MaxPool2D(2,2))
    self.add(Flatten())

    self.add(Dense(4096,activation='relu'))
    self.add(Dropout(0.5))
    
    self.add(Dense(4096,activation='relu'))
    self.add(Dropout(0.5))

    self.add(Dense(1000,activation='softmax'))

    self.optimizer=keras.optimizers.Adam(learning_rate=0.001)
    

    self.compile(optimizer=self.optimizer,loss='sparse_categorical_crossentropy',metrics='accuracy')



In [116]:
model=VGG16()
model.summary()

Model: "vg_g16_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_234 (Conv2D)          (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_235 (Conv2D)          (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_90 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_236 (Conv2D)          (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_237 (Conv2D)          (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_91 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_238 (Conv2D)          (None, 56, 56, 256)       29

In [117]:
early=keras.callbacks.EarlyStopping(patience=20)
checkpoint=keras.callbacks.ModelCheckpoint('vgg16.h5',save_best_only=True)

history=model.fit(train_ds,validation_data=val_ds,epochs=100,
                  callbacks=[early,checkpoint])

Epoch 1/100
704/704 [==============================] - 179s 252ms/step - loss: 13.7253 - accuracy: 0.1745 - val_loss: 1.5824 - val_accuracy: 0.4254
Epoch 2/100
704/704 [==============================] - 176s 250ms/step - loss: 1.5895 - accuracy: 0.4278 - val_loss: 1.2447 - val_accuracy: 0.5660
Epoch 3/100
704/704 [==============================] - 176s 250ms/step - loss: 1.1735 - accuracy: 0.5845 - val_loss: 0.9782 - val_accuracy: 0.6642
Epoch 4/100
704/704 [==============================] - 176s 250ms/step - loss: 0.9397 - accuracy: 0.6744 - val_loss: 1.0060 - val_accuracy: 0.6464
Epoch 5/100
704/704 [==============================] - 176s 250ms/step - loss: 0.7788 - accuracy: 0.7330 - val_loss: 0.9494 - val_accuracy: 0.6820
Epoch 6/100
704/704 [==============================] - 176s 250ms/step - loss: 0.6486 - accuracy: 0.7758 - val_loss: 0.8966 - val_accuracy: 0.7132
Epoch 7/100
704/704 [==============================] - 176s 250ms/step - loss: 0.5462 - accuracy: 0.8090 - val_loss: 